In [1]:
from simpletransformers.classification import ClassificationModel
import pandas as pd
import numpy as np
import time

In [4]:
def preproc(filename):
    data = pd.read_excel(filename)
    data = data[data.Frame.notna()]
    data = data[data.Text.notna()]
    text = list(data.Text)
    labels = list(data.Frame)
    final = []
    for i in range(len(text)):
        t = str(text[i])
        s = labels[i]
        if 'economic' == s:
            s = 0
        elif 'health' == s:
            s = 1
        else:
            s = 2
        final.append([t,s])
    return pd.DataFrame(final, columns=['text', 'labels'])

In [5]:
# Train and Evaluation data needs to be in a Pandas Dataframe of two columns. The first column is the text with type str, and the second column is the label with type int.
train = preproc('../../Dataset/Data/Frame/Train - labels approximated.xlsx')
# train = preproc('../../Dataset/Data/Frame/Train - clean.xlsx')
test = preproc('../../Dataset/Data/Frame/Test - clean.xlsx')

In [6]:
# Create a ClassificationModel
model_type = 'base'
model = ClassificationModel('roberta', 'roberta-{}'.format(model_type), num_labels=3, use_cuda=False) # You can set class weights by using the optional weight argument

In [7]:
# Train the model
b = time.time()
model.train_model(train, verbose=1)
e = time.time()
print(e-b)

Running loss: 1.017034

/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha)


Running loss: 0.278027

/home/roguehydra/.local/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Running loss: 0.265806


/home/roguehydra/.local/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)



886.3053059577942


In [8]:
# load model
# model = ClassificationModel(
#     "roberta",
#     "../../Demo application/best_frame/outputs/",
#     use_cuda=False
# )

In [9]:
b = time.time()
predictions, _ = model.predict(list(test.text))
e = time.time()
print(round(e-b,2))
labels = list(test.labels)
t = 0
for i in range(len(predictions)):
    if labels[i] == predictions[i]:
        t += 1
print('Acc: {}'.format(round(t/len(labels)*100,2)))


92.88
Acc: 59.5


In [21]:
for i in range(len(predictions)):
    pred = predictions[i]
    label = list(test.labels)[i]
    if pred != label:
        print('Prediction: {}\nActual: {}\nText:'.format(pred, label))
        print(test.text[i],'\n')

Prediction: 0
Actual: 2
Text:
these huge resources of natural gas , the cleanest - burning hydrocarbon , have the potential to help meet growing energy demand . 

Prediction: 1
Actual: 2
Text:
the gas production itself is not clean and safe . 

Prediction: 2
Actual: 1
Text:
we shine a light on the growing body of research showing dangerous levels of toxic air pollution near fracking sites , as well as evidence of threats from groundwater and surface - water contamination . 

Prediction: 2
Actual: 1
Text:
shale gas extraction can potentially affect the environment , health and the living environment . for example , chemicals could spread in the soil and groundwater in the event of leaks and substances could also end up in the air . 

Prediction: 2
Actual: 1
Text:
new report highlights dangers of unconventional fuel to environment and health . 

Prediction: 2
Actual: 1
Text:
' fracking ' , the process used to extract shale gas , is an unambiguously high - risk activity that threatens hum